In [1]:
import pandas as pd

df = pd.read_csv("../data/train.csv")
print(len(df))

2800


### 한자 탐색

In [2]:
import pandas as pd
import re
from collections import Counter

# 한자 추출 함수 정의
def extract_hanza(text):
    # CJK 통합 한자 범위(U+4E00 ~ U+9FFF)에 해당하는 문자 추출
    return "".join(re.findall("[\u4e00-\u9fff]", text))

# 문자 빈도 축정
def get_char_frequency(text_list):
    # 모든 문자를 하나의 리스트로 합치기
    chars = [char for text in text_list for char in text]
    # Counter를 사용하여 빈도 계산
    char_counts = Counter(chars)
    # 빈도순으로 정렬
    return pd.Series(char_counts).sort_values(ascending=False)

In [3]:
# 한자 빈도수 측정
df["hanza"] = df["text"].apply(extract_hanza)

char_frequency = get_char_frequency(df["hanza"])
print(char_frequency)
hanza = list(char_frequency.keys())
print(len(hanza), hanza)

美    79
北    55
中    43
朴    36
靑    24
日    21
與    20
文     9
英     7
野     6
佛     5
伊     5
獨     4
反     4
前     3
軍     3
硏     3
對     2
外     2
社     2
黃     2
亞     2
韓     2
株     1
車     1
崔     1
院     1
金     1
丁     1
小     1
和     1
企     1
安     1
展     1
檢     1
親     1
銀     1
證     1
先     1
父     1
南     1
詩     1
家     1
大     1
印     1
阿     1
故     1
州     1
重     1
dtype: int64
49 ['美', '北', '中', '朴', '靑', '日', '與', '文', '英', '野', '佛', '伊', '獨', '反', '前', '軍', '硏', '對', '外', '社', '黃', '亞', '韓', '株', '車', '崔', '院', '金', '丁', '小', '和', '企', '安', '展', '檢', '親', '銀', '證', '先', '父', '南', '詩', '家', '大', '印', '阿', '故', '州', '重']


In [4]:
df_hanza = df[df["hanza"].str.len()!=0]
print(len(df_hanza))
df_hanza = df_hanza[["ID", "text", "target"]]
#df_hanza.to_csv("hanza.csv",index=False)
df_hanza.head()

313


,ID,text,target
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
50,ynat-v1_train_00050,"美MBA[여성 비율x계속 x가4주$E19a대 U입생 중Ym,%",6
58,ynat-v1_train_00058,한#M2 !는 유`8 치료제 오B솔 美 임{ 3a 본격화,6


### 한자 전부 제거

In [5]:
def remove_chars_from_text(df, char_list):
    pattern = "[" + re.escape("".join(char_list)) + "]"
    df["filtered_text"] = df["filtered_text"].str.replace(pattern, "", regex=True)
    return df

In [6]:
df["filtered_text"] = df["text"].copy()
df = remove_chars_from_text(df, hanza)
df = df.drop(columns=["hanza"])
df.head()

,ID,text,target,filtered_text
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰.국DLwo 로L3한N% 회장 2 T0&}송=
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,"m 김정) 자주통일 새,?r열1나가야1보"
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,pI대선I앞두고 R2fr단 발] $비해 감시 강화


### 자주 쓰이는 특수문자 탐색

In [7]:
#영어, 숫자, 한글 제외
def extract_special(text):
    return "".join(re.findall(r"[^a-zA-Z0-9\sㄱ-ㅎㅏ-ㅣ가-힣]", text))

In [8]:
# 특수문자 빈도수 측정
special = df["filtered_text"].apply(extract_special)
char_frequency = get_char_frequency(special)
filtered_chars = char_frequency[char_frequency >= 100]
zazu_special = list(filtered_chars.keys())
print(len(zazu_special),zazu_special)

34 ['…', '.', '·', '%', '"', '-', '(', '|', '?', ',', '}', ':', '&', '_', '{', '~', '#', '\\', '*', ')', '$', '=', '+', '`', ';', "'", '!', '@', '<', '/', '>', '[', ']', '^']


### Case1. 말줄임표는 띄어쓰기로 치환

In [9]:
# df["filtered_text"] = df["filtered_text"].str.replace("…", " ")
# # df["filtered_text"] = df["filtered_text"].str.replace(
# #     "|".join(["…", "·", "/"]), " ", regex=True
# # )

### Case2. 100번 이상 사용된 특수문자 제거

In [10]:
df = remove_chars_from_text(df, zazu_special)
df["filtered_list"] = df["filtered_text"].apply(str.split)
df.head()

,ID,text,target,filtered_text,filtered_list
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정i 파1 미사z KT 이용기간 2e 단 Q분종U2보,"[정i, 파1, 미사z, KT, 이용기간, 2e, 단, Q분종U2보]"
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰국DLwo 로L3한N 회장 2 T0송,"[K찰국DLwo, 로L3한N, 회장, 2, T0송]"
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,m 김정 자주통일 새r열1나가야1보,"[m, 김정, 자주통일, 새r열1나가야1보]"
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,"[갤노트8, 주말, 27만대, 개통시장은, 불법, 보조금, 얼룩]"
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,pI대선I앞두고 R2fr단 발 비해 감시 강화,"[pI대선I앞두고, R2fr단, 발, 비해, 감시, 강화]"


### 형태소 분석기를 통한 한글+영문+숫자+특문 혼합 단어 탐지

In [11]:
# !pip install KoNLPy

In [12]:
# 분석기 임포트
from konlpy.tag import *

# 각 분석기 객체 생성
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [13]:
df["hannanum"] = df["filtered_list"].apply(lambda li: [hannanum.pos(e) for e in li])
df["kkma"] = df["filtered_list"].apply(lambda li: [kkma.pos(e) for e in li])
df["komoran"] = df["filtered_list"].apply(lambda li: [komoran.pos(e) for e in li])
df["okt"] = df["filtered_list"].apply(lambda li: [okt.pos(e) for e in li])

In [14]:
from collections import Counter

# 문장에서 조건 탐지
def check_tags_count_V1(morpheme_lists):
    # 단어에서 조건 탐지
    def check_tags_count_word(sublist):
        tag_counts = Counter(tag for _, tag in sublist)

        alpha = tag_counts["Alpha"]
        punctuation = tag_counts["Punctuation"]
        number = tag_counts["Number"]
        rest = sum(tag_counts.values()) - (alpha + punctuation + number)

        # 조건 리스트
        conditions = [
            alpha >= 1 and punctuation >= 1 and number >= 1,
            alpha >= 2,
            punctuation >= 2,
            # number >= 2,
        ]

        return any(conditions)

    # 하나의 단어라도 조건을 만족하면 True
    return any(check_tags_count_word(sublist) for sublist in morpheme_lists)


# 데이터프레임에 적용
df["check"] = df["okt"].apply(check_tags_count_V1)

In [15]:
df_morph_condition_V1 = df[df["check"]==True]
df_morph_condition_V1 = df_morph_condition_V1[["ID", "target", "text"]]
df_morph_condition_V1.to_csv("df_morph_condition_V1.csv",index=False)
print(len(df_morph_condition_V1))
df_morph_condition_V1.head()

1120


,ID,target,text
0,ynat-v1_train_00000,4,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
1,ynat-v1_train_00001,3,K찰.국DLwo 로L3한N% 회장 2 T0&}송=
4,ynat-v1_train_00004,6,pI美대선I앞두고 R2fr단 발] $비해 감시 강화
10,ynat-v1_train_00010,5,oi 매력 R모h츠a열#w3약 >l·주가 고Q/진
18,ynat-v1_train_00018,0,개R전 연w정연H 작가


### morph 패턴 V1 기반 1118개 실제 노이즈 탐지, 2개 오탐지

In [16]:
df = df[df["check"] == False]
len(df)

1680

In [17]:
# 문장 통째로 탐색
def check_tags_count_V2(morpheme_lists):
    # 태그 카운트
    all_tags = [tag for sublist in morpheme_lists for _, tag in sublist]
    tag_counts = Counter(all_tags)

    alpha = tag_counts["Alpha"]
    punctuation = tag_counts["Punctuation"]
    number = tag_counts["Number"]
    rest = sum(tag_counts.values()) - (alpha + punctuation + number)

    # 조건 리스트
    conditions = []
    # conditions.append(punctuation >= 1)
    conditions.append(alpha + punctuation + number >= 6)
    return all(conditions)

# 데이터프레임에 적용
df["check"] = df["okt"].apply(check_tags_count_V2)

In [18]:
df_morph_condition_V2 = df[df["check"] == True]
df_morph_condition_V2 = df_morph_condition_V2[["ID", "target", "text"]]
df_morph_condition_V2.to_csv("df_morph_condition_V2.csv", index=False)
print(len(df_morph_condition_V2))
df_morph_condition_V2.head()

101


,ID,target,text
13,ynat-v1_train_00013,4,아이`XSI수리0* b대`…맥3 디dF레< 41/'
14,ynat-v1_train_00014,2,"문/인 당2 4nS 민관2동7사위 /""X보 철거tt"
54,ynat-v1_train_00054,4,"찍W ,fK는 즐거m T졌다…5개 카메: LG V40 z큐"
58,ynat-v1_train_00058,6,한#M2 !는 유`8 치료제 오B솔 美 임{ 3a 본격화
94,ynat-v1_train_00094,1,"멀티골 j순형 u로축구 pB그1 1""\운드FXVP"


### morph 패턴 V2 기반 96개 실제 노이즈 탐지, 5개 오탐지

In [19]:
df = df[df["check"] == False]
len(df)

1579

### 한글 샌드위치 패턴
한글 사이에 낀 문자 중 (한글, 영어 대문자, 숫자, 관계 표현 특수문자)를 제외한 문자들 탐지

In [20]:
def detect_sandwich_pattern(text_list):
    return any(
        bool(re.search(r"[가-힣]+[^A-Z가-힣0-9ㆍ><∼→←↑↓↔]+[가-힣]+", str(item)))
        for item in text_list
    )

df["sandwich"] = df["filtered_list"].apply(detect_sandwich_pattern)
sandwich_df = df[df["sandwich"]==True]
not_sandwich_df = df[df["sandwich"] == False]

In [21]:
print(len(sandwich_df),len(not_sandwich_df))

121 1458


In [22]:
sandwich_df = sandwich_df[["ID", "target", "text"]]
sandwich_df.to_csv("sandwich_df.csv", index=False)

### 샌드위치 패턴 기반 121개 실제 노이즈 탐지. 0개 오탐지.

In [23]:
df = not_sandwich_df
len(df)

1458

### 문자 비율 기반 노이즈 탐색

In [24]:
import re

# 한글.숫자,영어 대문자 기반의 노이즈 탐색
def calculate_noise_ratio(text):
    if len(text)==0:
        return 0
    return round((len(re.findall(r"[^A-Z0-9가-힣\s]", text)) / len(text)), 4)

In [25]:
df.loc[:, "noise_ratio"] = df["filtered_text"].apply(calculate_noise_ratio)

In [26]:
high_special_character = df[df["noise_ratio"] >= 0.09]
print(len(high_special_character))
high_special_character = high_special_character.sort_values(
    by=["noise_ratio"], axis=0, ascending=True
)

high_special_character = high_special_character[["ID", "target", "text"]]
high_special_character = high_special_character.sort_values(by="ID", ascending=True)
high_special_character.to_csv("high_special_character.csv", index=False)
high_special_character.head()

93


,ID,target,text
6,ynat-v1_train_00006,1,프로야구~롯TKIAs광주 경기 y천취소
30,ynat-v1_train_00030,4,해외로밍 m금폭탄 n동차단 더 빨$진다
53,ynat-v1_train_00053,3,코로나 r대^등교)모습
101,ynat-v1_train_00101,1,~학농구리- 8일 고려대중앙대 경pt 개막
136,ynat-v1_train_00136,2,xW리 a)엔 예비후0V사전여론조사 결과 유출 논c


In [27]:
less_special_character = df[df["noise_ratio"]<0.09]
print(len(less_special_character))
less_special_character = less_special_character.sort_values(
    by=["noise_ratio"], axis=0, ascending=False
)
less_special_character = less_special_character[["ID", "target", "text"]]
less_special_character.to_csv("rest.csv", index=False)
less_special_character.head()

1365


,ID,target,text
1201,ynat-v1_train_01201,2,여야 ~종인 개헌특l 제안에 ~갈v -응&험로 예'
615,ynat-v1_train_00615,4,"티맥, 인d·AI a업지능으로 즈니(·생활 혁신"
2604,ynat-v1_train_02604,6,브_질A언론 {세프 V통. 탄핵rM능성 더 커j
2592,ynat-v1_train_02592,2,[ '대통령재벌 총수 u공& 면담 경위 수사종b
2473,ynat-v1_train_02473,1,v준우 \인 성적 필요 *어…5위 F[만 g각한다


### 문자 비율 기반 93개 실제 노이즈 탐지. 4개 오탐지

In [28]:
df = df[df["noise_ratio"] < 0.09]
len(df)

1365

In [29]:
df.head()

,ID,text,target,filtered_text,filtered_list,hannanum,kkma,komoran,okt,check,sandwich,noise_ratio
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,"[갤노트8, 주말, 27만대, 개통시장은, 불법, 보조금, 얼룩]","[[(갤노트8, N)], [(주말, N)], [(27만대, N)], [(개통시장, ...","[[(개, VV), (ㄹ, ETD), (노트, NNG), (8, NR)], [(주말...","[[(개, VV), (ㄹ, ETM), (노트, NNP), (8, SN)], [(주말...","[[(갤, Verb), (노트, Noun), (8, Number)], [(주말, N...",False,False,0.0
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0,성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다,"[성인, 6명, 중, 1명꼴, 배우자연인, 빚, 떠안은, 적, 있다]","[[(성인, N)], [(6명, N)], [(중, N)], [(1명, N), (꼴,...","[[(성인, NNG)], [(6, NR), (명, NNM)], [(중, NNG)],...","[[(성인, NNG)], [(6, SN), (명, NNB)], [(중, NNB)],...","[[(성인, Noun)], [(6, Number), (명, Noun)], [(중, ...",False,False,0.0
7,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,4,아가메즈 33득점 우리카드 KB손해보험 완파3위 굳,"[아가메즈, 33득점, 우리카드, KB손해보험, 완파3위, 굳]","[[(아가메즈, N)], [(33득점, N)], [(우리카드, N)], [(KB, ...","[[(아가, NNG), (메, NNG), (즈, UN)], [(33, NR), (득...","[[(아가메즈, NA)], [(33, SN), (득점, NNP)], [(우리카드, ...","[[(아가, Noun), (메, Noun), (즈, Modifier)], [(33,...",False,False,0.0
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6,대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,"[대통령, 얼마나, 많이, 놀라셨어요경주, 지진현장, 방문종합]","[[(대통령, N)], [(얼마나, M)], [(많, P), (이, X)], [(놀...","[[(대통령, NNG)], [(얼마나, MAG)], [(많이, MAG)], [(놀라...","[[(대통령, NNG)], [(얼마나, MAG)], [(많이, MAG)], [(놀라...","[[(대통령, Noun)], [(얼마나, Noun)], [(많이, Adverb)],...",False,False,0.0
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,"[듀얼심, 아이폰, 하반기, 출시설, 솔솔알뜰폰, 기대감]","[[(듀얼심, N)], [(아이폰, N)], [(하반기, N)], [(출시설, N)...","[[(듀얼, NNG), (심, NNG)], [(아이, NNG), (폰, NNG)],...","[[(듀얼심, NA)], [(아이폰, NNP)], [(하반기, NNG)], [(출시...","[[(듀얼, Noun), (심, Noun)], [(아이폰, Noun)], [(하반기...",False,False,0.0


## 연속 특수문자 패턴으로 탐지

In [34]:
import re

# 특수문자 연속으로 나오는 패턴 중 실제로 사용되는 의미있는 패턴
df.loc[:, "filtered_text"] = df["text"].copy()
df.loc[:, "filtered_text"] = df["filtered_text"].str.replace("…", " ")
df.loc[:, "filtered_text"] = df["filtered_text"].str.replace("...", "")
df.loc[:, "filtered_text"] = df["filtered_text"].str.replace("..", "")
df.loc[:, "filtered_text"] = df["filtered_text"].str.replace("%↑", "")
df.loc[:, "filtered_text"] = df["filtered_text"].str.replace("%↓", "")
df.loc[:, "filtered_text"] = df["filtered_text"].str.replace("%→", "")
df.loc[:, "filtered_text"] = df["filtered_text"].str.replace("%←", "")

# 띄어쓰기를 제외한 특수문자가 2회 이상 연속이면 True
def find_back_to_back_special(text):
    return bool(re.search(r"[^A-Za-z0-9가-힣\s\u4e00-\u9fff]{2,}", text))


df.loc[:, "special"] = df["filtered_text"].apply(find_back_to_back_special)
back_to_back_special = df[df["special"] == True]
print(len(back_to_back_special))
back_to_back_special = back_to_back_special[["ID", "target", "text"]]
back_to_back_special.to_csv("back_to_back_special.csv", index=False)

39


In [39]:
df = df[df["special"] == False]
print(len(df))
rule_based_not_noise = df[["ID", "target", "text"]]
rule_based_not_noise.to_csv("rule_based_not_noise.csv", index=False)

1326


### 연속 특수문자 기반 39개 실제 노이즈 탐지. 0개 오탐지

In [38]:
dfs = [
    pd.read_csv("df_morph_condition_V1.csv"),
    pd.read_csv("df_morph_condition_V2.csv"),
    pd.read_csv("sandwich_df.csv"),
    pd.read_csv("high_special_character.csv"),
    pd.read_csv("back_to_back_special.csv"),
]

rule_based_noise = pd.concat(dfs)
print(len(rule_based_noise))
rule_based_noise = rule_based_noise[["ID", "target", "text"]]
rule_based_noise = rule_based_noise.sort_values(by="ID", ascending=True)
rule_based_noise.to_csv("rule_based_noise.csv", index=False)

1474


# 모델 기반 노이즈 탐지
1. 현재 노이즈 탐지한 데이터 1474개, 남은 데이터 1326개 존재한다.
2. 1474개 라벨 1(노이즈)을 부여하고, 남은 데이터 1326개 라벨 0(비 노이즈)를 부여하고 두개의 셋으로 쪼갠다.
3. 라벨 1 + 라벨 0 절반을 학습 셋, 나머지 라벨 0 절반을 테스트 셋으로 만든다. (라벨0 + 라벨1-V1)+(라벨1-V2)와 (라벨0 + 라벨1-V2)+(라벨1-V1) 이렇게 두 쌍을 만들 수 있다. 
4. 두번의 학습&추론 과정으로 남은 데이터셋에 대한 노이즈 탐색을 수행할 수 있다.

In [46]:
rule_based_not_noise.loc[:, "label"] = 0
rule_based_noise.loc[:, "label"] = 1

# 추론할 데이터
rule_based_not_noise_V1 = rule_based_not_noise[:663].copy()
rule_based_not_noise_V2 = rule_based_not_noise[663:].copy()

# 훈련에 사용할 데이터
nd_train_V1 = pd.concat([rule_based_noise, rule_based_not_noise_V1])
nd_train_V2 = pd.concat([rule_based_noise, rule_based_not_noise_V2])

In [47]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
import os


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


# Set seed before any other operations
set_seed(42)  # You can change this seed value as needed


class NoiseTextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=64):
        self.texts = df["text"].values
        self.labels = df["label"].values
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }


def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)


def main(df_train, df_test, output_path):
    # Set seed at the start of main function as well
    set_seed(42)

    # 모델과 토크나이저 초기화
    model_name = "klue/bert-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # 데이터셋과 데이터로더 설정
    dataset = NoiseTextDataset(df_train, tokenizer)
    train_loader = DataLoader(
        dataset,
        batch_size=2,
        shuffle=True,
        worker_init_fn=lambda worker_id: np.random.seed(42),
    )

    # 학습 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

    # 학습 실행
    num_epochs = 3
    for epoch in range(num_epochs):
        train_loss = train_model(model, train_loader, criterion, optimizer, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

    # 예측 함수
    def predict_text(text):
        model.eval()
        encoding = tokenizer(
            text,
            add_special_tokens=True,
            max_length=64,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        input_ids = encoding["input_ids"].to(device)
        attention_mask = encoding["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            _, predicted = torch.max(predictions, dim=1)

        return 1 if predicted.item() == 1 else 0

    # 테스트 데이터에 대한 예측 결과 저장
    predictions = []
    for test_text in df_test["text"]:
        pred = predict_text(test_text)
        predictions.append(pred)

    # 예측 결과를 데이터프레임에 추가
    df_test["predicted"] = predictions

    # 결과 저장
    df_test.to_csv(output_path, index=False)
    print(f"Results saved to {output_path}")

    # 예측 결과 요약 출력
    print("\nPrediction Summary:")
    print(f"Total samples: {len(df_test)}")
    print(f"Predicted noise texts: {sum(predictions)}")
    print(f"Predicted normal texts: {len(predictions) - sum(predictions)}")


main(nd_train_V1, rule_based_not_noise_V2, "noise_detect_result_1.csv")
main(nd_train_V2, rule_based_not_noise_V1, "noise_detect_result_2.csv")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.1655
Epoch 2/3, Loss: 0.0883
Epoch 3/3, Loss: 0.0678
Results saved to noise_detect_result_1.csv

Prediction Summary:
Total samples: 663
Predicted noise texts: 48
Predicted normal texts: 615


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.1518
Epoch 2/3, Loss: 0.0988
Epoch 3/3, Loss: 0.0524
Results saved to noise_detect_result_2.csv

Prediction Summary:
Total samples: 663
Predicted noise texts: 45
Predicted normal texts: 618


In [48]:
df_model_predict_1 = pd.read_csv("./noise_detect_result_1.csv")
df_model_predict_2 = pd.read_csv("./noise_detect_result_2.csv")
df_model_predict = pd.concat([df_model_predict_1, df_model_predict_2])

In [49]:
df_model_based_noise = df_model_predict[df_model_predict["predicted"] == 1]
df_model_based_noise= df_model_based_noise[["ID","target","text"]]
print(len(df_model_based_noise))
df_model_based_noise = df_model_based_noise.sort_values(by="ID", ascending=True)
df_model_based_noise.to_csv("df_model_based_noise.csv", index=False)

93


In [53]:
df_model_based_not_noise = df_model_predict[df_model_predict["predicted"] == 0]
df_model_based_not_noise = df_model_based_not_noise[["ID", "target", "text"]]
print(len(df_model_based_not_noise))
df_model_based_not_noise = df_model_based_not_noise.sort_values(by="ID", ascending=True)
df_model_based_not_noise.to_csv("df_model_based_not_noise.csv", index=False)

1233


### 모델 기반 91개 실제 노이즈 탐지. 2개 오탐지

# 최종 결과: 오탐지 데이터 (13개)
- 노이즈 데이터 1567개 (실제 노이즈 1554개 + 비노이즈 데이터 13개)  
- 나머지 비 노이즈 데이터 1233개(노이즈 46개 + 비노이즈 1182개)  

### morph_condition V1 (2개)
```
ynat-v1_train_00858,5,한국MS 윈도10 IoT 에디션 출시…B2B 공략
ynat-v1_train_01717,6,中 베이징서 H7N9 조류인플루엔자 환자 또 발생
```

### morph_condition V2 (5개)
```
ynat-v1_train_00982,4,회전 카메라 탑재한 갤럭시A80 SKT 단독출시…59만9천500원
ynat-v1_train_01178,1,통신3사 5G 가입자 경쟁 불붙었다…LG V50 지원금 최대 77만원
ynat-v1_train_01225,5,세계 첫 5G폰 갤럭시S10 5G 출고가 139만7천원 확정종합
ynat-v1_train_01776,5,LGU 5G SA 상용화 준비…SA 기술 NSA 코어 장비에 연동 검증
ynat-v1_train_01873,3,LG전자 스마트폰사업 10분기 적자…3Q 영업손실 3천753억종합
```

### sandwich (0개)
```
```

### 문자 비율 기반 정규표현식 (4개)
```
ynat-v1_train_00191,4,MLB.com 다저스 3년 연속 WS 유력하지만 우승은 글쎄…
ynat-v1_train_00741,6,게시판 SKT AI 콘퍼런스 ai.x 2019 개최
ynat-v1_train_02249,0,클린턴재단 후폭풍인가…힐러리 vs 트럼프 지지율 3%p로 좁혀져
ynat-v1_train_02399,2,재단법인화 tbs 초대 대표에 이강택 현 교통방송 대표
```

### 연속 특수문자 기반 (0개)

### 모델 기반 탐지 (2개)
```
ynat-v1_train_02448,5,애플 국내서도 아이폰 보상판매…아이폰XR 69만원부터
ynat-v1_train_02472,5,상상인 3분4 영업익 436QK…66% 증가
```


# 하나의 데이터로 합치기

In [63]:
import pandas as pd

# CSV 파일 읽기
df1 = pd.concat(
    [pd.read_csv("rule_based_noise.csv"), pd.read_csv("df_model_based_noise.csv")]
)
df2 = pd.read_csv("df_model_based_not_noise.csv")

# 각각의 데이터프레임에 is_noise 컬럼 추가
df1["is_noise"] = 1
df2["is_noise"] = 0

# 두 데이터프레임 합치기
df_combined = pd.concat([df1, df2], ignore_index=True)
df_combined = df_combined[["ID","text","target","is_noise"]]
df_combined = df_combined.sort_values(by="ID", ascending=True)

# 결과 저장
df_combined.to_csv("noise_detected_train.csv", index=False)

In [64]:
len(df_combined)

2800

In [65]:
df_combined.value_counts("is_noise")

is_noise
1    1567
0    1233
Name: count, dtype: int64